<a href="https://colab.research.google.com/github/Rweiber/Check-News/blob/main/Check_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai
!pip install -q ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00


In [11]:
# Importa as bibliotecas necessárias
from pathlib import Path
import hashlib
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
from google.colab import userdata

api_key = userdata.get("SECRET_KEY")
# Configura a API Key do Google Generative AI
genai.configure(api_key=api_key)

# Define as configurações do modelo Gemini Pro
generation_config = {
    "temperature": 1, # Controla a criatividade do modelo (valores mais altos = mais criativo)
    "top_p": 0.95, # Limita a probabilidade das próximas palavras
    "top_k": 0, # Não limita o número de palavras consideradas
    "max_output_tokens": 8192, # Define o limite máximo de tokens na resposta
}

# Define as configurações de segurança para o modelo
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT", # Assédio
        "threshold": "BLOCK_MEDIUM_AND_ABOVE", # Bloqueia respostas com nível médio ou alto de assédio
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH", # Discurso de ódio
        "threshold": "BLOCK_MEDIUM_AND_ABOVE", # Bloqueia respostas com nível médio ou alto de discurso de ódio
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", # Conteúdo sexualmente explícito
        "threshold": "BLOCK_MEDIUM_AND_ABOVE", # Bloqueia respostas com nível médio ou alto de conteúdo sexualmente explícito
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT", # Conteúdo perigoso
        "threshold": "BLOCK_MEDIUM_AND_ABOVE", # Bloqueia respostas com nível médio ou alto de conteúdo perigoso
    },
]

# Define a instrução inicial para o modelo
system_instruction = "Haja como um investigador de noticias, que pesquisa nas fontes de noticias mais qualificadas do brasil e retorno uma porcentagem do quão verdadeira é a noticia, tambem preciso que descreva a imagem, por completa no inicio da resposta, no final sempre coloque as fontes de onde pesquisou e o o link da fonte"

# Cria uma instância do modelo Gemini Pro
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    system_instruction=system_instruction,
    safety_settings=safety_settings,
)

# Cria os widgets
upload_button = widgets.FileUpload(accept="image/*", multiple=False, description="Enviar imagem/print para verificação", layout=widgets.Layout(width='70%'))
output_text = widgets.Output()

# Define a função que será chamada quando o usuário carregar uma imagem
def on_upload_change(change):
    if change["new"]:
        # Obtém o nome do arquivo e o conteúdo da imagem carregada
        uploaded_filename = list(upload_button.value.keys())[0]
        uploaded_content = upload_button.value[uploaded_filename]["content"]

        # Cria um arquivo temporário para armazenar o conteúdo da imagem
        image_path = Path(uploaded_filename)
        with open(image_path, "wb") as f:
            f.write(uploaded_content)

        # Executa a análise da imagem
        with output_text:
            print("Analisando a imagem...")

            # Inicia uma conversa com o modelo Gemini Pro
            convo = model.start_chat(history=[
                {
                    "role": "user", # Define o papel do usuário
                    "parts": [genai.upload_file(image_path)] # Carrega a imagem para o modelo
                },
            ])

            # Envia a mensagem para o modelo, solicitando a análise da imagem
            response = convo.send_message(
                "que pesquisa nas fontes de noticias mais qualificadas do brasil e retorno uma porcentagem do quão verdadeira é a noticia"
            )

            # Divide a resposta em blocos de 25 MB
            chunks = [response.text[i:i+25000000] for i in range(0, len(response.text), 25000000)]

            # Salva cada bloco em um arquivo separado
            for i, chunk in enumerate(chunks):
              with open(f"resposta_parte{i+1}.txt", "w") as f:
                f.write(chunk)

            # Exibe a resposta do modelo no widget de saída em Markdown
            display(Markdown(f"---\n**Resposta:** {response.text}"))

        # Remove o arquivo temporário da imagem
        image_path.unlink()

# Configura a função on_upload_change para ser chamada quando o valor do upload_button mudar
upload_button.observe(on_upload_change, names="value")

# Exibe os widgets na tela
display(upload_button)
display(output_text)

FileUpload(value={}, accept='image/*', description='Enviar imagem/print para verificação', layout=Layout(width…

Output()